# Yearly Production of Vegetables and Animal Products per Country

In [1]:
# The usual imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import json

In [2]:
# Constants
DATA_PATH = '../data/'
GENERATED_DATA_PATH = '../generated/'

## Introduction
### Vegetables
To have an estimate of the total amount of vegetables produced by a country on a yearly basis, we will use the 'Production - Crops' data from the FAOSTAT dataset. The vegetable production is split in 10 categories, for which we only take into account the vegetables that are used for food (not *feed*).
These categories are:

1. Cereals: http://www.fao.org/waicent/faoinfo/economic/faodef/fdef01e.htm
2. Roots and Tubers: http://www.fao.org/waicent/faoinfo/economic/faodef/fdef02e.htm
3. Sugar Crops and Sweeteners: http://www.fao.org/waicent/faoinfo/economic/faodef/fdef03e.htm
4. Pulses: http://www.fao.org/waicent/faoinfo/economic/faodef/fdef04e.htm
5. Nuts: http://www.fao.org/waicent/faoinfo/economic/faodef/fdef05e.htm
6. Oil Crops: http://www.fao.org/waicent/faoinfo/economic/faodef/fdef06e.htm
7. Vegetables: http://www.fao.org/waicent/faoinfo/economic/faodef/fdef07e.htm
8. Fruits: http://www.fao.org/waicent/faoinfo/economic/faodef/fdef08e.htm
9. Spices: http://www.fao.org/waicent/faoinfo/economic/faodef/fdef10e.htm
10. Stimulant Crops: http://www.fao.org/waicent/faoinfo/economic/faodef/fdef12e.htm

For these categories, we closely looked at the available documentation to determine the relevant items. Derived products are thus not included.

### Animals
For the animals we consider three categories:

1. Meat: http://www.fao.org/waicent/faoinfo/economic/faodef/fdef17e.htm
2. Fodder: http://www.fao.org/waicent/faoinfo/economic/faodef/fdef11e.htm
3. Eggs: http://www.fao.org/waicent/faoinfo/economic/faodef/fdef18e.htm#18.6
4. Milk: http://www.fao.org/waicent/faoinfo/economic/faodef/fdef18e.htm#18.1
    
For this the vegetable data is combined with the 'Production - Livestock Primary' dataset. Again, we avoid any overlaps, to get an estimate which is as closely as possible to the actual production within each category.

## Data Preperation

In [3]:
# Load the data
# Production - Crops dataset
production_crops = pd.read_csv(os.path.join(DATA_PATH, "Production_Crops_E_All_Data.csv"), encoding='latin-1')
production_livestock = pd.read_csv(os.path.join(DATA_PATH, "Production_LivestockPrimary_E_All_Data.csv"), encoding="latin-1")

# Load the items for each category
with open(os.path.join(DATA_PATH, "production_classifications.json"), "r") as f:
    classifications = json.load(f)
    
display(production_crops.head())
display(production_livestock.head())

,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,...,Y2013,Y2013F,Y2014,Y2014F,Y2015,Y2015F,Y2016,Y2016F,Y2017,Y2017F
0,2,Afghanistan,221,"Almonds, with shell",5312,Area harvested,ha,NaN,NaN,NaN,...,14114.0,NaN,13703.0,NaN,14676.0,NaN,19481.0,NaN,19793.0,NaN
1,2,Afghanistan,221,"Almonds, with shell",5419,Yield,hg/ha,NaN,NaN,NaN,...,29910.0,Fc,19996.0,Fc,16521.0,Fc,16859.0,Fc,13788.0,Fc
2,2,Afghanistan,221,"Almonds, with shell",5510,Production,tonnes,NaN,NaN,NaN,...,42215.0,NaN,27400.0,NaN,24246.0,NaN,32843.0,NaN,27291.0,NaN
3,2,Afghanistan,711,"Anise, badian, fennel, coriander",5312,Area harvested,ha,NaN,M,NaN,...,18500.0,F,30000.0,F,25000.0,F,26019.0,Im,28873.0,Im
4,2,Afghanistan,711,"Anise, badian, fennel, coriander",5419,Yield,hg/ha,NaN,NaN,NaN,...,6757.0,Fc,7167.0,Fc,7200.0,Fc,6923.0,Fc,6830.0,Fc


,Area Code,Area,Item Code,Item,Element Code,Element,Unit,Y1961,Y1961F,Y1962,...,Y2013,Y2013F,Y2014,Y2014F,Y2015,Y2015F,Y2016,Y2016F,Y2017,Y2017F
0,2,Afghanistan,1062,"Eggs, hen, in shell",5313,Laying,1000 Head,4000.0,F,4400.0,...,9500.0,F,9500.0,F,9337.0,Im,9369.0,Im,10688.0,F
1,2,Afghanistan,1062,"Eggs, hen, in shell",5410,Yield,100mg/An,25000.0,Fc,25000.0,...,18947.0,Fc,19474.0,Fc,21253.0,Fc,21263.0,Fc,18713.0,Fc
2,2,Afghanistan,1062,"Eggs, hen, in shell",5510,Production,tonnes,10000.0,F,11000.0,...,18000.0,F,18500.0,F,19844.0,Im,19921.0,Im,20000.0,F
3,2,Afghanistan,1067,"Eggs, hen, in shell (number)",5513,Production,1000 No,200000.0,F,220000.0,...,360000.0,F,370000.0,F,396880.0,Im,398420.0,Im,400000.0,F
4,2,Afghanistan,919,"Hides, cattle, fresh",5320,Producing Animals/Slaughtered,Head,360000.0,F,384000.0,...,744500.0,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Combine both datasets
production = production_crops.append(production_livestock)

In [5]:
# Drop the flag columns for every year
production = production.drop([f"Y{i}F" for i in range(1961, 2018)], axis=1)

# Transform the dataset to contain seperate columns for the value and the year, instead of seperate columnts for 1961 - 2017
production = production.melt(id_vars=["Area Code", "Area", "Item", "Item Code", "Element Code", "Element", "Unit"],
                             value_name="Value",
                             var_name="Year")

# Clean the year column
production["Year"] = pd.to_numeric(production["Year"].str[1:])

# We are only interested in the Production in tonnes TODO: ARE WE? Area harvested/number of animals could be interesting?
production = production.query("`Element` == 'Production'")
# Drop the Element, Element Code and Unit columns: aren't of interest anymore
production = production.drop(["Element Code", "Element", "Unit"], axis=1)

production.head()

,Area Code,Area,Item,Item Code,Year,Value
2,2,Afghanistan,"Almonds, with shell",221,1961,NaN
5,2,Afghanistan,"Anise, badian, fennel, coriander",711,1961,NaN
8,2,Afghanistan,Apples,515,1961,15100.0
11,2,Afghanistan,Apricots,526,1961,32000.0
14,2,Afghanistan,Barley,44,1961,378000.0


In [30]:
# Get a seperate dataframe for each category

def items_to_dataset(items, name):
    """
    Transform the items list to a dataset that contains the sum of the items
    for every country per year, using the production_crops dataset
    """
    # transform the items to integers
    items = list(map(int, items))

    # select the data that contains the items we want to sum together
    selection = production[production["Item Code"].isin(items)]
    
    # drop the item and item code columns: will be summed away
    selection = selection.drop(["Item", "Item Code"], axis=1)
    
    # sum per area, per year
    selection = selection.groupby(["Area Code", "Area", "Year"], as_index=False).sum()
    
    # rename
    return selection.rename({"Value": name}, axis=1)
    
# create the dataset for the vegetable and meat categories
production_aggr = None
for key, val in list(classifications["Plants"].items()) + list(classifications["Animals"].items()):
    if production_aggr is None:
        production_aggr = items_to_dataset(val, key)
    else:
        production_aggr = production_aggr.merge(items_to_dataset(val, key), on=["Area", "Area Code", "Year"], how="outer")

In [32]:
production_aggr.query("`Area` == 'Belgium'").tail()

,Area Code,Area,Year,Cereals,Roots,Sugar,Pulses,Nuts,Oilcrops,Vegetables,Fruit,Spices,Stimulants,Milk,Eggs,Meat
10825,255,Belgium,2013,3097497.0,3428000.0,4809300.0,4250.0,410.0,68300.0,2074064.0,582634.0,0.0,0.0,3483230.0,173530.0,1774712.0
10826,255,Belgium,2014,3162852.0,4380556.0,4790700.0,5888.0,405.0,65500.0,1892951.0,748224.0,0.0,0.0,3701535.0,151870.0,1814781.0
10827,255,Belgium,2015,3163733.0,3689994.0,4532561.0,7127.0,400.0,58744.0,1529210.0,723887.0,0.0,0.0,4006912.0,148025.0,1850880.0
10828,255,Belgium,2016,2287774.0,3402787.0,4021110.0,6583.0,395.0,48452.0,1752544.0,630677.0,0.0,0.0,3906695.0,154600.0,1805539.0
10829,255,Belgium,2017,2764392.0,4416665.0,5941783.0,6867.0,390.0,53812.0,1805035.0,450261.0,0.0,0.0,4058322.0,138320.0,1794958.0
